# Final Exam
## Octavio Villalaz

In [18]:
#install.packages(c("caret", "e1071", "pROC"))
library(caret)
library(pROC)

diabetes_data <- read.csv("diabetes.csv")
columns_with_zero <- c('Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI')

# Replace zero values with the median of the column
for (col in columns_with_zero) {
  diabetes_data[[col]][diabetes_data[[col]] == 0] <- median(diabetes_data[[col]][diabetes_data[[col]] != 0])
}

preproc <- preProcess(diabetes_data[, -9], method = c("center", "scale"))
diabetes_data[, -9] <- predict(preproc, diabetes_data[, -9])

diabetes_data$Outcome <- factor(diabetes_data$Outcome, levels = c(0, 1), labels = c("No", "Yes"))

In [22]:
# 10-fold cross-validation
control <- trainControl(method = "cv", number = 10, classProbs = TRUE, summaryFunction = twoClassSummary)

#KNN
model_knn <- train(as.factor(Outcome) ~ ., data = diabetes_data, method = "knn", trControl = control, metric = "ROC")
print(model_knn)

predictions <- predict(model_knn, diabetes_data)
prob_predictions <- predict(model_knn, diabetes_data, type = "prob")[, 2]

# Evaluate
accuracy <- mean(predictions == diabetes_data$Outcome)
confusion <- confusionMatrix(as.factor(predictions), as.factor(diabetes_data$Outcome))
precision <- confusion$byClass["Pos Pred Value"]
recall <- confusion$byClass["Sensitivity"]
f1 <- 2 * (precision * recall) / (precision + recall)
roc_auc <- roc(diabetes_data$Outcome, prob_predictions)$auc


cat("K-Nearest Neighbors Model Performance (Cross-Validation):\n")
cat("Accuracy:", accuracy, "\n")
cat("Precision:", precision, "\n")
cat("Recall:", recall, "\n")
cat("F1 Score:", f1, "\n")
cat("ROC-AUC:", roc_auc, "\n")

k-Nearest Neighbors 

768 samples
  8 predictor
  2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 691, 691, 691, 692, 692, 691, ... 
Resampling results across tuning parameters:

  k  ROC        Sens   Spec     
  5  0.7916410  0.818  0.5783476
  7  0.8017778  0.830  0.6008547
  9  0.8131011  0.840  0.5935897

ROC was used to select the optimal model using the largest value.
The final value used for the model was k = 9.


Setting levels: control = No, case = Yes

Setting direction: controls < cases



K-Nearest Neighbors Model Performance (Cross-Validation):
Accuracy: 0.8072917 
Precision: 0.8371648 
Recall: 0.874 
F1 Score: 0.8551859 
ROC-AUC: 0.8822873 


Accuracy: The model correctly predicts the outcome 81% of the time.
<br>
Precision: When the model predicts a patient has diabetes, it is correct 84% of the time.
<br>
Recall: The model identifies 87% of the patients who actually have diabetes.
<br>
F1 Score: The harmonic mean of precision and recall, which balances the two, is 0.86.
<br>
ROC-AUC: The model has a strong ability to distinguish between patients with and without diabetes, with an area under the ROC curve of 0.88.
<br>
Cross-Validation: By using 10-fold cross-validation, the model's performance metrics are more robust and less likely to be biased due to the specific data split.
<br>
Balanced Performance: The model shows a good balance between precision and recall, which is crucial for medical diagnoses where both false positives and false negatives can have significant consequences.